我们首先关注具有单隐藏层的多层感知机

In [2]:
import torch
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[0.0761],
        [0.0667]], grad_fn=<AddmmBackward>)

参数访问

In [3]:
print(net[2].state_dict())#网络第2层的权重的状态

OrderedDict([('weight', tensor([[ 0.3164, -0.0901,  0.0410, -0.0354,  0.1634,  0.1115,  0.0531,  0.0623]])), ('bias', tensor([-0.1808]))])


目标参数

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1808], requires_grad=True)
tensor([-0.1808])


In [5]:
net[2].weight.grad==None

True

In [7]:
net[0].weight.grad==None

True

一次性访问所有参数

In [8]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [9]:
net.state_dict()['2.bias'].data

tensor([-0.1808])

从嵌套块收集参数

In [11]:
def block1():
  return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
  net=nn.Sequential()
  for i in range(4):
    net.add_module(f'block{i}',block1())
  return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.2144],
        [-0.2143]], grad_fn=<AddmmBackward>)

我们已经设计了网络，让我们看看它是如何组织的

In [13]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


内置初始化

In [15]:
def init_normal(m):
  if type(m)==nn.Linear:
    nn.init.normal_(m.weight,mean=0,std=0.01)
    nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data

(tensor([-0.0147,  0.0278, -0.0063,  0.0225]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [16]:
def init_constant(m):#m就是一个module
  if type(m)==nn.Linear:
    nn.init.constant_(m.weight,1)
    nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data

(tensor([1., 1., 1., 1.]), tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

对某些块应用不同的初始化方法

In [17]:
def xavier(m):
  if type(m)==nn.Linear:
    nn.init.xavier_uniform_(m.weight)

def init_42(m):
  if type(m)==nn.Linear:
    nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3984,  0.3230,  0.3497, -0.5205])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [19]:
def my_init(m):
  if type(m)==nn.Linear:
    print(
        "Init",
        *[(name,param.shape) for name ,param in m.named_parameters()][0])
    nn.init.uniform_(m.weight,-10,10)
    m.weight.data*=m.weight.data.abs()>=5 #保留计算后大于5的数，否则为0

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 9.4433, -6.3696, -6.0950,  0.0000],
        [ 0.0000, -7.6996,  0.0000,  9.7001]], grad_fn=<SliceBackward>)

In [20]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000, -5.3696, -5.0950,  1.0000])

参数绑定

In [21]:
shared=nn.Linear(8,8)#这个层的参数都相同
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))
net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
